<a href="https://colab.research.google.com/github/liangchingyun/A-Convolution-Module-For-Variable-Input-Channels/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [46]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
